In [2]:
from datetime import datetime
import time
import os

import numpy as np
import pandas as pd

import vectorbt as vbt

In [3]:
start = '2019-01-01 UTC'  # crypto is in UTC
end = '2020-01-01 UTC'
data = vbt.YFData.download('BTC-USD', start=start, end=end).get('Close')


In [14]:
fast_ma = vbt.MA.run(btc_price, 10, short_name='fast')
slow_ma = vbt.MA.run(btc_price, 20, short_name='slow')


time_sum = 0
n = 100
for i in range(n):
    st = time.perf_counter_ns()
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    pf = vbt.Portfolio.from_signals(btc_price, entries, exits)
    et = time.perf_counter_ns()
    time_sum += et - st

avg_time_millis = time_sum / n / 1e6
tr = pf.total_return()

print(f"Total return: {tr:.3%}")
print(f"Time elapsed Avg: {avg_time_millis:.3f} ms")

Total return: 63.519%
Time elapsed Avg: 18.029 ms


In [30]:
df = pd.DataFrame(btc_price)
df["ma_fast"] = df["Close"].rolling(10).mean()
df["ma_slow"] = df["Close"].rolling(20).mean()
df.index = df.index.astype('int64')
df.dropna(inplace=True)
df

,Close,ma_fast,ma_slow
Date,,,
1547942400000000000,3601.013672,3655.933936,3786.643591
1548028800000000000,3576.032471,3644.800635,3773.269214
1548115200000000000,3604.577148,3639.128247,3756.327600
1548201600000000000,3585.123047,3642.345239,3743.746692
1548288000000000000,3600.865479,3631.826562,3730.904089
...,...,...,...
1577404800000000000,7290.088379,7288.358252,7235.358203
1577491200000000000,7317.990234,7292.477002,7223.040454
1577577600000000000,7422.652832,7314.457861,7224.128125


In [31]:
output_path = os.path.join(os.getcwd(), 'data_fast.h5')
if os.path.exists(output_path):
    os.remove(output_path)

ticker = "BTCUSD"
with h5py.File(output_path, "a") as file:
        stock_data = df.to_numpy()
        index = df.index.map(lambda timestamp: int(timestamp)).to_numpy()
        dataset = file.create_dataset(f"{ticker}/data", data=stock_data)
        file.create_dataset(
                f"{ticker}/datetime",
                data = index,
        )
        for column in df.columns:
            dataset.attrs[column] = column

In [8]:
(time_sum / n) * 1e4

54.108643531799316